**Step 1.** Frame extraction from input video 'mrbeantrim.mp4'

In [3]:
import cv2
import math
# input path to video
videoFile = str(input("Enter input to video:"))
# output path for frames
framesFolder = str(input("Enter output path for frames:"))
cap = cv2.VideoCapture(videoFile)
# find frame rate of video
frameRate = cap.get(5)
while cap.isOpened() :
#     current frame number
    frameId = cap.get(1)
    ret, frame = cap.read()
    if (ret != True) :
        break
    if int(frameId) % math.floor(frameRate) == 0 :
        filename = framesFolder + "/image_" + str(int(frameId)) + ".jpg"
#         cv2.imshow(filename, frame)
        cv2.imwrite(filename, frame)
cap.release()
print "Done!"

Enter input to video:"/Users/arin/Desktop/final_yr_project/frame_extract/mrbeantrim.mp4"
Enter output path for frames:"/Users/arin/Desktop/final_yr_project/frame_extract/frames"
Done!


summarzation algorithm testing

In [1]:
from collections import Counter
import nltk
import re

def commonprefix(m):
    if not m:
        return ''
#     testing
#     for line in m:
#         print line, len(line)
    s1 = min(m)
    print s1
    s2 = max(m)
    print s2
    for i, c in enumerate(s1):
        if c != s2[i]:
            return s1[:i]
    return s1

def makesent(text):
    sents = filter(lambda x: x!= '', [line.strip() for line in text.split('.')])
    print sents
    tags, target_sents_NN, target_sents_VBG, req_lines = list(), list(), list(), list()
    dummylines, objects = list(), list()
    lookup, target = dict(), dict()
    
    for line in sents:
        parse = nltk.word_tokenize(line)
        tags.append(nltk.pos_tag(parse))
    counts = Counter([item for line in tags for item in line])
#     print tags
    
    targetVBG = None
    targetNN = None
    
    for item in counts.most_common():
        if item[0][1] == 'NN':
            targetNN = item[0][0]
            break
        
    for item in counts.most_common() :
        if item[0][1] == 'DT':
            targetDT = item[0][0]
            break
            
    for key, value in counts.items():
        lookup.setdefault(key[1],[]).append(key[0])
    
    for line in tags:
        for word in line:
            if word[0] == targetNN:
                target_sents_NN.append(line)
                print line
                print '------------------'
#     targetVBG = str(0) 
#     targetNN = str(0)
    
    for item in Counter([item for line in target_sents_NN for item in line]).most_common():
        if item[0][1] == 'VBG':
            targetVBG = str(item[0][0])
            break
            
    if targetVBG is None:
        print "no targetVBG"
        return;
    elif targetNN is None:
        print "no targetNN"
        return;
    else :
        for line in target_sents_NN:
            if (targetNN, 'NN') in line and (targetVBG, 'VBG') in line:
                target_sents_VBG.append(line)
            
    if 'VBZ' not in lookup:
        print "cannot display"
    else:
        VBZ_list = lookup['VBZ']
        print VBZ_list
#     if len(VBZ_list) == 0:
#         print "Cannot display"
#         return
#     else:
        target_prefix = [' '.join([targetNN, targetVBG])]
        target_prefix.append(' '.join([targetDT, targetNN, targetVBG]))
        print target_prefix
        for vbz in VBZ_list:
            target_prefix.append(' '.join([targetDT, targetNN, vbz, targetVBG]))
            print target_prefix
        
        for line in sents:
            for tp in target_prefix:
                if line.startswith(tp):
                    req_lines.append(line)
                    print req_lines
                    
        for line in req_lines:
            dummylines.append(line.replace(targetDT + ' ', ''))
        print dummylines
        
        prefix = commonprefix(dummylines)
        
        for line in dummylines:
            line = re.sub(prefix, '',line) 
            if 'and ' in line:
                diff = list()
                diff = line.split(' and ')
                for d in diff:
                    objects.append(d)
                    break
            else:
                objects.append(line)
            print objects
            
        obj = list(set(objects))
        print obj
        obj.sort()
        obj.sort(key=len)
        print obj
        
        req_objects = filter(lambda x: [x for i in obj if x in i and x != i] == [], obj)
        final_text = ' '.join([targetDT, prefix]) + ', '.join(req_objects[:-1]) + ' and ' + req_objects[-1]
        
        print final_text.capitalize()
        
        
if __name__ == '__main__':
#     text = raw_input("Enter your text:")
#     print "Your text:" + text
    text = "the man has glasses. a man wearing a white shirt. a white door. a pair of blue jeans. the shirt is black. a white shirt on a man. man wearing a suit and tie. a man in a white shirt. a glass of wine. man has white shirt. a wooden table. a white box. a white box on the table. a man in a black shirt. a man wearing a tie. the mans hair is black. a wooden door. man wearing a black suit. a picture of a woman. a person standing in the background. a white napkin in the background. black hat on mans head. man wearing a white shirt. a chair with a white seat. a pair of brown pants. the man is wearing a tie."
    makesent(text)

['the man has glasses', 'a man wearing a white shirt', 'a white door', 'a pair of blue jeans', 'the shirt is black', 'a white shirt on a man', 'man wearing a suit and tie', 'a man in a white shirt', 'a glass of wine', 'man has white shirt', 'a wooden table', 'a white box', 'a white box on the table', 'a man in a black shirt', 'a man wearing a tie', 'the mans hair is black', 'a wooden door', 'man wearing a black suit', 'a picture of a woman', 'a person standing in the background', 'a white napkin in the background', 'black hat on mans head', 'man wearing a white shirt', 'a chair with a white seat', 'a pair of brown pants', 'the man is wearing a tie']
[('the', 'DT'), ('man', 'NN'), ('has', 'VBZ'), ('glasses', 'NNS')]
------------------
[('a', 'DT'), ('man', 'NN'), ('wearing', 'VBG'), ('a', 'DT'), ('white', 'JJ'), ('shirt', 'NN')]
------------------
[('a', 'DT'), ('white', 'JJ'), ('shirt', 'NN'), ('on', 'IN'), ('a', 'DT'), ('man', 'NN')]
------------------
[('man', 'NN'), ('wearing', 'VBG

testing summarization

In [1]:
from collections import Counter
import nltk
import re
import json

def commonprefix(m):
    if not m:
        return ''
#     testing
#     for line in m:
#         print line, len(line)
    s1 = min(m)
#     print s1
    s2 = max(m)
#     print s2
    for i, c in enumerate(s1):
        if c != s2[i]:
            return s1[:i]
    return s1

def makesent(text):
    sents = list(filter(lambda x: x!= '', [line.strip() for line in text.split('.')]))
#     print (sents)
#     print ('-------------')
    tags, target_sents_NN, target_sents_VBG, req_lines = list(), list(), list(), list()
    dummylines, objects = list(), list()
    lookup, target = dict(), dict()
    
    for line in sents:
        parse = nltk.word_tokenize(line)
        tags.append(nltk.pos_tag(parse))
    counts = Counter([item for line in tags for item in line])
#     print tags
    
    targetVBG = None
    targetNN = None
    for item in counts.most_common():
        if item[0][1] == 'NN':
            targetNN = item[0][0]
            break
        
    for item in counts.most_common() :
        if item[0][1] == 'DT':
            targetDT = item[0][0]
            break
            
    for key, value in counts.items():
        lookup.setdefault(key[1],[]).append(key[0])
    
    for line in tags:
        for word in line:
            if word[0] == targetNN:
                target_sents_NN.append(line)
#                 print line
#                 print '------------------'
#     targetVBG = str(0) 
#     targetNN = str(0)
    
    for item in Counter([item for line in target_sents_NN for item in line]).most_common():
        if item[0][1] == 'VBG':
            targetVBG = str(item[0][0])
            break
            
    if targetVBG is None:
#         print("no targetVBG")
        return;
    elif targetNN is None:
#         print("no targetNN")
        return;
    else :
        for line in target_sents_NN:
            if (targetNN, 'NN') in line and (targetVBG, 'VBG') in line:
                target_sents_VBG.append(line)
            
    if 'VBZ' not in lookup:
#         print("cannot display")
        pass
    else:
        VBZ_list = lookup['VBZ']
#         print VBZ_list
#     if len(VBZ_list) == 0:
#         print "Cannot display"
#         return
#     else:
        target_prefix = [' '.join([targetNN, targetVBG])]
        target_prefix.append(' '.join([targetDT, targetNN, targetVBG]))
#         print target_prefix
        for vbz in VBZ_list:
            target_prefix.append(' '.join([targetDT, targetNN, vbz, targetVBG]))
#             print target_prefix
        
        for line in sents:
            for tp in target_prefix:
                if line.startswith(tp):
                    req_lines.append(line)
#                     print req_lines
                    
        for line in req_lines:
            dummylines.append(line.replace(targetDT + ' ', ''))
#         print dummylines
        
        prefix = commonprefix(dummylines)
        
        for line in dummylines:
            line = re.sub(prefix, '',line) 
            if 'and ' in line:
                diff = list()
                diff = line.split(' and ')
                for d in diff:
                    objects.append(d)
                    break
            else:
                objects.append(line)
#             print objects
            
        obj = list(set(objects))
#         print obj
        obj.sort()
        obj.sort(key=len)
#         print(obj)
#         print('-----------')
        req_objects = []
#         req_objects.append(str(filter(lambda x: [x for i in obj if x in i and x != i] == [], obj)))
        req_objects = list(filter(lambda x: [x for i in obj if x in i and x != i] == [], obj))
#         print(len(req_objects))
        if len(req_objects) > 0:
            final_text = ' '.join([targetDT, prefix]) + ', '.join(req_objects[:-1]) + ' and ' + req_objects[-1]
            mysummary.append(final_text.capitalize())
        
        
if __name__ == '__main__':
    mysummary = []
    mytext = json.load(open('/Users/arin/Desktop/final_yr_project/frame_extract/results/mrbeantrimresults/textsent.json'))
#     print(mytext[0])
    for item in mytext:
        makesent(item)
    print(str(mysummary))
#     text = raw_input("Enter your text:")
#     print "Your text:" + text
#     text = "the man has glasses. a man wearing a white shirt. a white door. a pair of blue jeans. the shirt is black. a white shirt on a man. man wearing a suit and tie. a man in a white shirt. a glass of wine. man has white shirt. a wooden table. a white box. a white box on the table. a man in a black shirt. a man wearing a tie. the mans hair is black. a wooden door. man wearing a black suit. a picture of a woman. a person standing in the background. a white napkin in the background. black hat on mans head. man wearing a white shirt. a chair with a white seat. a pair of brown pants. the man is wearing a tie."
#     makesent(text)

['A man wearing tie, suit, watch, black shirt, white shirt and black jacket', 'The man is wearing a white shirt and ', 'A man wearing hat, glasses and white shirt', 'A man wearing watch and ', 'A man wearing tie, suit, watch, black shirt, white shirt and black jacket', 'The man is wearing a white shirt and ', 'A man wearing hat, glasses and white shirt', 'A man wearing watch and ', 'A man wearing tie, blue shirt and white shirt', 'A man wearing black suit, ue shirt and ack shirt', 'A man wearing tie, suit, watch, black shirt, white shirt and black jacket', 'The man is wearing a white shirt and ', 'A man wearing hat, glasses and white shirt', 'A man wearing watch and ', 'A man wearing tie, blue shirt and white shirt', 'A man wearing black suit, ue shirt and ack shirt', 'A man wearing tie, suit and blue shirt', 'A man wearing black tie and suit', 'A man wearing tie, suit, watch, black shirt, white shirt and black jacket', 'The man is wearing a white shirt and ', 'A man wearing hat, glass